Collect word frequency from Microsoft N-Gram service
=============================

In [1]:
import csv
from collections import Counter
import MicrosoftNgram as mn
from time import sleep
import string

In [2]:
# Load files
inputFile = 'data/cat_desc_wn.csv'
headers = None
descCnt = Counter()
with open(inputFile) as fin:
    r = csv.reader(fin)
    for row in r:
        if headers is None:
            headers = row
            print headers
        else:
            descCnt[row[1]] += int(row[2])

['category_id', 'description', 'count']


In [3]:
threshold = 10
queryList = [w for w, v in descCnt.most_common() if v >= threshold]
print len(queryList)

3058


In [4]:
token = 'df5dc471-6880-4e5c-86bf-0049f44df1e8'
lookup = mn.LookupService(token)
print lookup.GetModel()
print lookup.GetJointProbability('class room')
print lookup.GetJointProbability('classroom')

bing-body/2013-12/3
-6.43
-4.57


In [5]:
wordLog10Freq = {}
wordLog10FreqFile = 'data/MsrWordLog10Freq.csv'
i = 0
while len(wordLog10Freq) < len(queryList):
    w = queryList[i]
    i = (i+1) % len(queryList)
    if w not in wordLog10Freq:
        try:
            wordLog10Freq[w] = lookup.GetJointProbability(w.replace('_', ' '))
            sleep(0.2)
            if len(wordLog10Freq) % 100 == 0:
                print len(wordLog10Freq), w, wordLog10Freq[w]
        except:
            sleep(10)
print set(wordLog10Freq.keys()) == set(queryList)
with open(wordLog10FreqFile, 'w') as fout:
    fw = csv.writer(fout)
    fw.writerow(['description', 'log10JointProb'])
    for w in queryList:
        fw.writerow([w, str(wordLog10Freq[w])])

100 animal -4.057
200 bathtub -5.439
300 fire -3.733
400 pier -5.186
500 washroom -6.026
600 control_center -5.912
700 showcase -4.806
800 engine -3.795
900 ballpark -5.98
1000 post -3.111
1100 chef -4.447
1200 workman -5.849
1300 bread -4.502
1400 dance_hall -6.609
1500 airport_terminal -6.433
1600 sand_trap -7.211
1700 solar_energy -5.437
1800 sandpit -6.966
1900 lava -5.414
2000 marching -5.442
2100 batting -5.2
2200 turtle -5.121
2300 award -4.162
2400 skater -5.717
2500 rome -4.698
2600 item -3.334
2700 field_work -6.286
2800 location -3.328
2900 handiwork -6.649
3000 pop -4.237
True


In [6]:
for w in queryList:
    if ',' in w:
        print 'ERROR', w
    elif any(c not in string.ascii_lowercase and c != ' ' and c != ',' for c in w.replace('_', ' ')):
        print w
googleNGramResp = {}

child's_room
merry-go-round
farmer's_market
baby's_room
child's_play


In [44]:
# Google N-Gram
import urllib
import urllib2
import json
baseURL='https://books.google.com/ngrams/graph?case_insensitive=on&year_start=1990&year_end=2008&corpus=15&smoothing=20&content='
discardChar='&.,'
dataLine = 'var data = ['
handleAposHTML = lambda s: s.replace('&#39;', "'")
def fetchGoogleNGram(wLst):
    wLst2 = [w.translate(None, discardChar).replace('_', ' ') for w in wLst]
    wParam = urllib.quote_plus(','.join(wLst2))
    resp = urllib2.urlopen(baseURL + wParam, timeout=10).read().split('\n')
    dataLines = [s for s in resp if s.strip().startswith(dataLine)]
    if len(dataLines) != 1:
        raise RuntimeError('Confused by response: ' + str(resp))
    data = dataLines[0].strip()[len(dataLine) - 1:-1]
    if len(data.strip()) == 0:
        data = []
    else:
        data = json.loads(data)
    ret = [0.0] * len(wLst)
    for d in data:
        if len(d) == 0:
            continue
        if d['parent'] != '':
            continue
        matching = [w for w in wLst2 if handleAposHTML(d['ngram']).replace(' ', '').lower()
                    .startswith(w.replace(' ', '').lower())]
        if len(matching) == 0:
            continue
        if len(matching) > 1:
            t = max(matching, key=len)
            if any(not t.startswith(s) for s in matching):
                raise RuntimeError('Confused by response: ' + str(matching)
                                   + ' | ' + d['ngram'] + ' | ' + str(dataLines))
            else:
                matching = [t]
        if len(matching) == 1:
            ret[wLst2.index(matching[0])] = d['timeseries'][-1]
    for i, x in enumerate(ret):
        if x == 0.0:
            print 'Not Found:', wLst[i], '|', wLst2[i]
    return ret

In [ ]:
batchSize = 10
i = 0
while len(googleNGramResp) < len(queryList):
    wLst = []
    while len(wLst) < min(batchSize, len(queryList) - len(googleNGramResp)):
        if queryList[i] not in googleNGramResp and queryList[i] not in wLst:
            wLst.append(queryList[i])
        i = (i + 1) % len(queryList)
    resp = None
    try:
        sleep(1)
        resp = fetchGoogleNGram(wLst)
    except Exception as e:
        sleep(10)
        print type(e)
        print e
    if resp is not None:
        for w, x in zip(wLst, resp):
            googleNGramResp[w] = x
        if len(googleNGramResp) % 10 == 0:
            print 'Success', len(googleNGramResp)

In [64]:
outputFile = 'data/GNGramFreq.csv'
with open(outputFile, 'w') as fout:
    fw = csv.writer(fout)
    fw.writerow(['description', 'JointProb'])
    for w in queryList:
        fw.writerow([w, str(googleNGramResp[w])])

In [56]:
for w in googleNGramResp.keys():
    if googleNGramResp[w] == 0:
        del googleNGramResp[w]

In [59]:
wLst = []
while len(wLst) < min(batchSize, len(queryList) - len(googleNGramResp)):
    if queryList[i] not in googleNGramResp and queryList[i] not in wLst:
        wLst.append(queryList[i])
    i = (i + 1) % len(queryList)
print wLst

['wasteyard']


In [62]:
googleNGramResp['wasteyard'] = 0.0
print googleNGramResp['wasteyard']

0.0


In [63]:
googleNGramResp["child's_play"] = 3.3154049737161905e-07
print googleNGramResp["child's_play"]

3.31540497372e-07
